In [4]:
import numpy as np
import torch as tc
import copy

In [5]:
def hosvd(x):
    """
    :param x: 待分解的张量
    :return G: 核张量
    :return U: 变换矩阵
    :return lm: 各个键约化矩阵的本征谱
    """
    ndim = x.ndim
    U = list()  # 用于存取各个变换矩阵
    lm = list()  # 用于存取各个键约化矩阵的本征谱
    x = tc.from_numpy(x)
    for n in range(ndim):
        index = list(range(ndim))
        index.pop(n)
        _mat = tc.tensordot(x, x, [index, index])
        _lm, _U = tc.symeig(_mat, eigenvectors=True)
        lm.append(_lm.numpy())
        U.append(_U)
    # 计算核张量
    G = tucker_product(x, U)
    U1 = [u.numpy() for u in U]
    return G, U1, lm


def tucker_product(x, U, dim=1):
    """
    :param x: 张量
    :param U: 变换矩阵
    :param dim: 收缩各个矩阵的第几个指标
    :return G: 返回Tucker乘积的结果
    """
    ndim = x.ndim
    if type(x) is not tc.Tensor:
        x = tc.from_numpy(x)
        
    U1 = list()
    for n in range(len(U)):
        if type(U[n]) is not tc.Tensor:
            U1.append(tc.from_numpy(U[n]))
        else:
            U1.append(U[n])
    
    ind_x = ''
    for n in range(ndim):
        ind_x += chr(97 + n)
    ind_x1 = ''
    for n in range(ndim):
        ind_x1 += chr(97 + ndim + n)
    contract_eq = copy.deepcopy(ind_x)
    for n in range(ndim):
        if dim == 0:
            contract_eq += ',' + ind_x[n] + ind_x1[n]
        else:
            contract_eq += ',' + ind_x1[n] + ind_x[n]
    contract_eq += '->' + ind_x1
    # print(x.shape, U[0].shape, U[1].shape, U[2].shape)
    # print(type(contract_eq), contract_eq)
    G = tc.einsum(contract_eq, [x] + U1)
    G = G.numpy()
    return G

In [6]:
tensor = np.random.randn(3, 4, 2)
Core, V, LM = hosvd(tensor)

print('检查Tucker分解等式是否成立：')
tensor1 = tucker_product(Core, V, dim=0)
error = np.linalg.norm(tensor - tensor1)
print('Tucker分解误差 = ' + str(error))

print('\n检查各个变换矩阵为正交阵：')
for n, v in enumerate(V):
    print('\n对于第' + str(n) +'个变换矩阵，VV.T = ')
    print(v.dot(v.T))

# ----------------------------------------------------
# 练习：
# 1. 编写任意给定四阶张量HOSVD的程序，并于jupyter notebook中任意阶张量的分解结果对比；
# 2. 编写程序验证核张量满足的性质。

检查Tucker分解等式是否成立：
Tucker分解误差 = 2.931448150165722e-15

检查各个变换矩阵为正交阵：

对于第0个变换矩阵，VV.T = 
[[ 1.00000000e+00 -1.67292532e-16 -1.27537875e-16]
 [-1.67292532e-16  1.00000000e+00  3.60919370e-17]
 [-1.27537875e-16  3.60919370e-17  1.00000000e+00]]

对于第1个变换矩阵，VV.T = 
[[ 1.00000000e+00  1.66533454e-16 -4.51028104e-16  9.71445147e-17]
 [ 1.66533454e-16  1.00000000e+00 -1.11022302e-16  2.08166817e-17]
 [-4.51028104e-16 -1.11022302e-16  1.00000000e+00  5.55111512e-17]
 [ 9.71445147e-17  2.08166817e-17  5.55111512e-17  1.00000000e+00]]

对于第2个变换矩阵，VV.T = 
[[ 1.00000000e+00 -2.01870893e-17]
 [-2.01870893e-17  1.00000000e+00]]
